# Types Spec - Framework-Agnostic Field Specification

The `types.spec` module provides framework-agnostic field specifications for type validation and schema definition:

**Core Components:**
- **Spec**: Framework-agnostic field spec combining base_type + metadata
- **CommonMeta**: Standard metadata keys (NAME, NULLABLE, LISTABLE, VALIDATOR, DEFAULT, DEFAULT_FACTORY)
- **Meta**: Immutable key-value pairs for extensible metadata

**Key Features:**
- Type annotation generation (plain and Annotated)
- Nullable/listable modifiers
- Default values and factory functions (sync/async)
- Validator integration
- Functional updates (immutable API)
- Thread-safe annotated type caching
- Integration with Operable for structured outputs

In [1]:
from dataclasses import dataclass

from lionherd_core.types import CommonMeta, Meta, Spec, Undefined

## 1. CommonMeta - Standard Metadata Keys

CommonMeta defines standard metadata keys used throughout lionherd for field specifications.

In [2]:
# CommonMeta enum provides standard keys
print("Standard metadata keys:")
for meta in CommonMeta:
    print(f"  {meta.name} = {meta.value}")

# Access as enum members
print(f"\nNAME key: {CommonMeta.NAME.value}")
print(f"NULLABLE key: {CommonMeta.NULLABLE.value}")
print(f"DEFAULT key: {CommonMeta.DEFAULT.value}")

Standard metadata keys:
  NAME = name
  NULLABLE = nullable
  LISTABLE = listable
  VALIDATOR = validator
  DEFAULT = default
  DEFAULT_FACTORY = default_factory

NAME key: name
NULLABLE key: nullable
DEFAULT key: default


In [3]:
# CommonMeta.prepare() validates metadata constraints
valid_meta = CommonMeta.prepare(name="user_id", nullable=True, default="default_value")
print(f"Valid metadata: {valid_meta}")

# Cannot have both default and default_factory
try:
    invalid = CommonMeta.prepare(default="value", default_factory=lambda: "factory_value")
except ExceptionGroup as eg:
    print(f"\n✓ Validation error: {eg.exceptions[0]}")

Valid metadata: (Meta(key='name', value='user_id'), Meta(key='nullable', value=True), Meta(key='default', value='default_value'))

✓ Validation error: Cannot provide both 'default' and 'default_factory'


## 2. Basic Spec Construction

Spec combines a base type with metadata to define field specifications.

In [4]:
# Simple spec with just a type
basic_spec = Spec(str)
print(f"Basic spec: {basic_spec}")
print(f"Base type: {basic_spec.base_type}")
print(f"Metadata: {basic_spec.metadata}")

Basic spec: Spec(base_type=<class 'str'>, metadata=())
Base type: <class 'str'>
Metadata: ()


In [5]:
# Spec with metadata via kwargs
named_spec = Spec(int, name="user_id", nullable=False)
print(f"Named spec: {named_spec}")
print(f"Metadata count: {len(named_spec.metadata)}")
for meta in named_spec.metadata:
    print(f"  {meta.key} = {meta.value}")

Named spec: Spec(base_type=<class 'int'>, metadata=(Meta(key='name', value='user_id'), Meta(key='nullable', value=False)))
Metadata count: 2
  name = user_id
  nullable = False


In [6]:
# Spec with Meta objects as args
meta_spec = Spec(
    str, Meta("name", "email"), Meta("nullable", True), Meta("description", "User email address")
)
print(f"Spec with Meta objects: {meta_spec.metadata}")

Spec with Meta objects: (Meta(key='name', value='email'), Meta(key='nullable', value=True), Meta(key='description', value='User email address'))


In [7]:
# Spec validates base_type is actually a type
try:
    invalid_spec = Spec("not_a_type", name="field")
except ValueError as e:
    print(f"✓ Type validation: {e}")

# Generic types work
list_spec = Spec(list[str], name="tags")
dict_spec = Spec(dict[str, int], name="counts")
print(f"\n✓ Generic types accepted: {list_spec.base_type}, {dict_spec.base_type}")

✓ Type validation: base_type must be a type or type annotation, got not_a_type

✓ Generic types accepted: list[str], dict[str, int]


## 3. Metadata Access Patterns

Spec provides multiple ways to access metadata: dict-like `[]`, `.get()`, and properties.

In [8]:
spec = Spec(str, name="username", nullable=True, default="anonymous", description="User login name")

# Dict-like access with []
print(f"Name via []: {spec['name']}")
print(f"Nullable via []: {spec['nullable']}")

# KeyError for missing keys
try:
    missing = spec["nonexistent"]
except KeyError as e:
    print(f"\n✓ KeyError for missing: {e}")

Name via []: username
Nullable via []: True

✓ KeyError for missing: "Metadata key 'nonexistent' undefined in Spec."


In [9]:
# .get() with default value
print(f"Name via .get(): {spec.get('name')}")
print(f"Missing with default: {spec.get('missing', 'default_value')}")

# .get() returns Undefined if key missing and no default
result = spec.get("nonexistent")
print(f"Missing without default: {result}")
print(f"Is Undefined: {result is Undefined}")

Name via .get(): username
Missing with default: default_value
Missing without default: Undefined
Is Undefined: True


In [10]:
# Custom metadata keys work too
custom_spec = Spec(int, name="count", min_value=0, max_value=100, unit="items")

print("Custom metadata:")
print(f"  min_value: {custom_spec['min_value']}")
print(f"  max_value: {custom_spec['max_value']}")
print(f"  unit: {custom_spec['unit']}")

Custom metadata:
  min_value: 0
  max_value: 100
  unit: items


## 4. Spec Properties

Spec provides convenience properties for common metadata: name, is_nullable, is_listable, default.

In [11]:
spec = Spec(str, name="email", nullable=True, listable=True, default="noreply@example.com")

# Property access
print(f"Name property: {spec.name}")
print(f"Is nullable: {spec.is_nullable}")
print(f"Is listable: {spec.is_listable}")
print(f"Default: {spec.default}")

Name property: email
Is nullable: True
Is listable: True
Default: noreply@example.com


In [12]:
# Properties return sensible defaults when metadata missing
minimal_spec = Spec(int)

print(f"Name (undefined): {minimal_spec.name}")
print(f"Is undefined: {minimal_spec.name is Undefined}")
print(f"Is nullable (missing = False): {minimal_spec.is_nullable}")
print(f"Is listable (missing = False): {minimal_spec.is_listable}")
print(f"Default (undefined): {minimal_spec.default}")

Name (undefined): Undefined
Is undefined: True
Is nullable (missing = False): False
Is listable (missing = False): False
Default (undefined): Undefined


In [13]:
# Nullable/listable are boolean properties - explicit check
nullable_spec = Spec(str, nullable=True)
not_nullable_spec = Spec(str, nullable=False)
no_nullable_spec = Spec(str)  # Missing = False

print(f"Explicitly nullable: {nullable_spec.is_nullable}")
print(f"Explicitly not nullable: {not_nullable_spec.is_nullable}")
print(f"Missing (treated as False): {no_nullable_spec.is_nullable}")

Explicitly nullable: True
Explicitly not nullable: False
Missing (treated as False): False


## 5. Default Values and Factories

Spec supports default values (static) and default factories (callable), including async factories.

In [14]:
# Static default value
static_spec = Spec(str, name="status", default="pending")

print(f"Has default: {static_spec.default is not Undefined}")
print(f"Default value: {static_spec.default}")
print(f"Has factory: {static_spec.has_default_factory}")

# Create default value
value = static_spec.create_default_value()
print(f"Created value: {value}")

Has default: True
Default value: pending
Has factory: False
Created value: pending


In [15]:
# Default factory (callable)
import datetime as dt

factory_spec = Spec(dt.datetime, name="created_at", default_factory=lambda: dt.datetime.now(dt.UTC))

print(f"Has factory: {factory_spec.has_default_factory}")
print(f"Is async factory: {factory_spec.has_async_default_factory}")

# Factory called each time
value1 = factory_spec.create_default_value()
import time

time.sleep(0.01)
value2 = factory_spec.create_default_value()

print(f"\nValue 1: {value1}")
print(f"Value 2: {value2}")
print(f"Different instances: {value1 != value2}")

Has factory: True
Is async factory: False

Value 1: 2025-11-09 05:55:30.735457+00:00
Value 2: 2025-11-09 05:55:30.748006+00:00
Different instances: True


In [16]:
# Async default factory
async def async_factory():
    """Simulate async default creation."""

    await asyncio.sleep(0.001)
    return "async_value"


# Warning issued for async factories
import warnings

with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    async_spec = Spec(str, name="async_field", default_factory=async_factory)
    if w:
        print(f"✓ Warning: {w[0].message}")

print(f"\nHas async factory: {async_spec.has_async_default_factory}")

✓ Warning: Async default factories are not yet fully supported by all adapters. Consider using sync factories for compatibility.

Has async factory: True


In [17]:
# Async factory requires acreate_default_value()
try:
    value = async_spec.create_default_value()  # Sync method fails
except ValueError as e:
    print(f"✓ Sync method rejects async factory: {e}")

# Use async method
import asyncio

value = await async_spec.acreate_default_value()
print(f"\nAsync created value: {value}")

# acreate_default_value works for sync factories too
sync_value = await factory_spec.acreate_default_value()
print(f"Sync factory via async method: {sync_value}")

✓ Sync method rejects async factory: Default factory is asynchronous; cannot create default synchronously. Use 'await spec.acreate_default_value()' instead.

Async created value: async_value
Sync factory via async method: 2025-11-09 05:55:30.755541+00:00


In [18]:
# No default - raises error
no_default_spec = Spec(str, name="required")

try:
    value = no_default_spec.create_default_value()
except ValueError as e:
    print(f"✓ No default error: {e}")

✓ No default error: No default value or factory defined in Spec.


## 6. Validators and Validation

Spec supports attaching validators - functions that validate field values.

In [19]:
# Single validator function
def validate_email(value: str) -> str:
    """Validate email format."""
    if "@" not in value:
        raise ValueError(f"Invalid email: {value}")
    return value


email_spec = Spec(str, name="email", validator=validate_email)

print(f"Has validator: {email_spec.get('validator') is not Undefined}")
print(f"Validator: {email_spec['validator']}")

# Validator would be called by Operable during validation
valid_email = email_spec["validator"]("user@example.com")
print(f"\n✓ Valid email: {valid_email}")

try:
    email_spec["validator"]("invalid_email")
except ValueError as e:
    print(f"✓ Validator caught error: {e}")

Has validator: True
Validator: <function validate_email at 0x10769c4a0>

✓ Valid email: user@example.com
✓ Validator caught error: Invalid email: invalid_email


In [20]:
# Multiple validators (list)
def not_empty(value: str) -> str:
    if not value.strip():
        raise ValueError("Value cannot be empty")
    return value


def min_length(value: str, min_len: int = 3) -> str:
    if len(value) < min_len:
        raise ValueError(f"Value must be at least {min_len} chars")
    return value


multi_validator_spec = Spec(str, name="username", validator=[not_empty, min_length])

validators = multi_validator_spec["validator"]
print(f"Validators: {validators}")
print(f"Count: {len(validators)}")

# Run all validators
value = "alice"
for validator in validators:
    value = validator(value)
print(f"\n✓ All validators passed: {value}")

Validators: [<function not_empty at 0x10769c680>, <function min_length at 0x10769c7c0>]
Count: 2

✓ All validators passed: alice


In [21]:
# Validator validation - must be callable
try:
    invalid = Spec(str, validator="not_a_function")
except ExceptionGroup as eg:
    print(f"✓ Validator validation: {eg.exceptions[0]}")

try:
    invalid = Spec(str, validator=[validate_email, "not_callable"])
except ExceptionGroup as eg:
    print(f"✓ List validator validation: {eg.exceptions[0]}")

✓ Validator validation: Validators must be a list of functions or a function
✓ List validator validation: Validators must be a list of functions or a function


## 7. Spec Transformations

Spec is immutable - transformations create new Spec instances with updated metadata.

In [22]:
# with_updates() - general update method
original = Spec(str, name="field", nullable=False)

updated = original.with_updates(nullable=True, description="Updated field")

print(f"Original nullable: {original.is_nullable}")
print(f"Updated nullable: {updated.is_nullable}")
print(f"Original description: {original.get('description')}")
print(f"Updated description: {updated.get('description')}")
print(f"\nDifferent instances: {original is not updated}")

Original nullable: False
Updated nullable: True
Original description: Undefined
Updated description: Updated field

Different instances: True


In [23]:
# as_nullable() - convenience for nullable=True
not_nullable = Spec(int, name="count")
nullable = not_nullable.as_nullable()

print(f"Original: nullable={not_nullable.is_nullable}")
print(f"Modified: nullable={nullable.is_nullable}")

# as_listable() - convenience for listable=True
scalar = Spec(str, name="tag")
listable = scalar.as_listable()

print(f"\nOriginal: listable={scalar.is_listable}")
print(f"Modified: listable={listable.is_listable}")

Original: nullable=False
Modified: nullable=True

Original: listable=False
Modified: listable=True


In [24]:
# with_default() - add default value or factory
no_default = Spec(str, name="status")

# Static value
with_static = no_default.with_default("pending")
print(f"Static default: {with_static.default}")
print(f"Has factory: {with_static.has_default_factory}")

# Callable automatically becomes factory
with_factory = no_default.with_default(lambda: "dynamic_value")
print(f"\nFactory default: {with_factory.default}")
print(f"Has factory: {with_factory.has_default_factory}")
print(f"Created value: {with_factory.create_default_value()}")

Static default: pending
Has factory: False

Factory default: <function <lambda> at 0x10769c860>
Has factory: True
Created value: dynamic_value


In [25]:
# with_validator() - add validator(s)
no_validator = Spec(int, name="age")


def validate_age(value: int) -> int:
    if value < 0 or value > 150:
        raise ValueError(f"Invalid age: {value}")
    return value


with_validator = no_validator.with_validator(validate_age)
print(f"Has validator: {with_validator.get('validator') is not Undefined}")

# Test validator
result = with_validator["validator"](30)
print(f"✓ Valid age: {result}")

try:
    with_validator["validator"](200)
except ValueError as e:
    print(f"✓ Invalid age caught: {e}")

Has validator: True
✓ Valid age: 30
✓ Invalid age caught: Invalid age: 200


In [26]:
# Chain transformations
base = Spec(str, name="email")

enhanced = (
    base.as_nullable()
    .with_default("noreply@example.com")
    .with_validator(validate_email)
    .with_updates(description="User email address")
)

print("Enhanced spec metadata:")
for meta in enhanced.metadata:
    print(f"  {meta.key} = {meta.value}")

Enhanced spec metadata:
  name = email
  nullable = True
  default = noreply@example.com
  validator = <function validate_email at 0x10769c4a0>
  description = User email address


## 8. Type Annotations

Spec generates type annotations in two forms: plain (`annotation`) and `Annotated` with metadata.

In [27]:
# Plain annotation - just the type with nullable/listable modifiers
simple = Spec(str, name="field")
print(f"Simple annotation: {simple.annotation}")

nullable = Spec(str, name="field", nullable=True)
print(f"Nullable annotation: {nullable.annotation}")

listable = Spec(int, name="field", listable=True)
print(f"Listable annotation: {listable.annotation}")

both = Spec(str, name="field", nullable=True, listable=True)
print(f"Nullable + listable: {both.annotation}")

Simple annotation: <class 'str'>
Nullable annotation: str | None
Listable annotation: list[int]
Nullable + listable: list[str] | None


In [28]:
# annotated() - Annotated[type, metadata...]
spec = Spec(str, name="email", nullable=False, description="User email", validator=validate_email)

annotated_type = spec.annotated()
print(f"Annotated type: {annotated_type}")

# Extract metadata from Annotated
import typing

if hasattr(typing, "get_args"):
    args = typing.get_args(annotated_type)
    print(f"\nAnnotated args count: {len(args)}")
    print(f"Base type: {args[0]}")
    print(f"Metadata items: {len(args) - 1}")

Annotated type: typing.Annotated[str, Meta(key='name', value='email'), Meta(key='nullable', value=False), Meta(key='description', value='User email'), Meta(key='validator', value=<function validate_email at 0x10769c4a0>)]

Annotated args count: 5
Base type: <class 'str'>
Metadata items: 4


In [29]:
# Nullable in annotated() uses union syntax
nullable_spec = Spec(int, name="count", nullable=True)
nullable_annotated = nullable_spec.annotated()
print(f"Nullable annotated: {nullable_annotated}")

# Get the actual union type
import typing

if hasattr(typing, "get_args"):
    args = typing.get_args(nullable_annotated)
    base_type = args[0] if args else nullable_annotated
    print(f"Base union: {base_type}")

Nullable annotated: typing.Annotated[int | None, Meta(key='name', value='count'), Meta(key='nullable', value=True)]
Base union: int | None


In [30]:
# annotated() uses thread-safe LRU cache
spec1 = Spec(str, name="field1", nullable=True)
spec2 = Spec(str, name="field1", nullable=True)  # Same type + metadata

ann1 = spec1.annotated()
ann2 = spec2.annotated()

# Cache returns same type object
print(f"Same cached type: {ann1 is ann2}")

# Different metadata = different type
spec3 = Spec(str, name="field2", nullable=True)  # Different name
ann3 = spec3.annotated()
print(f"Different metadata: {ann1 is not ann3}")

Same cached type: True
Different metadata: True


In [31]:
# No base_type (None or Undefined) becomes Any
no_type = Spec(None, name="any_field")
print(f"None base_type annotation: {no_type.annotation}")

from typing import Any

print(f"Is Any: {no_type.annotation is Any}")

None base_type annotation: typing.Any
Is Any: True


## 9. Metadata Dictionary Extraction

Spec provides `metadict()` to extract metadata as a dictionary with filtering options.

In [32]:
spec = Spec(
    str,
    name="username",
    nullable=True,
    default="anonymous",
    description="User login name",
    min_length=3,
    max_length=50,
)

# Get all metadata as dict
all_meta = spec.metadict()
print(f"All metadata: {all_meta}")
print(f"Keys: {list(all_meta.keys())}")

All metadata: {'name': 'username', 'nullable': True, 'default': 'anonymous', 'description': 'User login name', 'min_length': 3, 'max_length': 50}
Keys: ['name', 'nullable', 'default', 'description', 'min_length', 'max_length']


In [33]:
# Exclude specific keys
exclude_default = spec.metadict(exclude={"default"})
print(f"Exclude default: {exclude_default}")
print(f"'default' in dict: {'default' in exclude_default}")

Exclude default: {'name': 'username', 'nullable': True, 'description': 'User login name', 'min_length': 3, 'max_length': 50}
'default' in dict: False


In [34]:
# Exclude common metadata (NAME, NULLABLE, LISTABLE, etc.)
custom_only = spec.metadict(exclude_common=True)
print(f"Custom metadata only: {custom_only}")
print("\nCommon keys excluded:")
for key in CommonMeta.allowed():
    print(f"  {key} in dict: {key in custom_only}")

Custom metadata only: {'description': 'User login name', 'min_length': 3, 'max_length': 50}

Common keys excluded:
  name in dict: False
  nullable in dict: False
  listable in dict: False
  validator in dict: False
  default in dict: False
  default_factory in dict: False


In [35]:
# Combine exclude and exclude_common
minimal = spec.metadict(exclude={"description"}, exclude_common=True)
print(f"Minimal metadata: {minimal}")
print(f"Only custom non-description: {list(minimal.keys())}")

Minimal metadata: {'min_length': 3, 'max_length': 50}
Only custom non-description: ['min_length', 'max_length']


## 10. Integration with Operable and Structured Outputs

Spec integrates with Operable to define structured output schemas for LLM operations.

In [36]:
# Define output schema using Spec
from dataclasses import field
from typing import ClassVar


@dataclass
class UserProfile:
    """Structured user profile output."""

    # Define specs as class variables
    _specs: ClassVar[dict[str, Spec]] = {
        "username": Spec(str, name="username", description="User login name", nullable=False),
        "email": Spec(
            str,
            name="email",
            description="User email address",
            nullable=True,
            validator=validate_email,
        ),
        "age": Spec(int, name="age", description="User age in years", nullable=True),
        "tags": Spec(
            str, name="tags", description="User tags", listable=True, default_factory=list
        ),
    }

    username: str = field(default=Undefined)
    email: str | None = field(default=None)
    age: int | None = field(default=None)
    tags: list[str] = field(default_factory=list)


# Access specs
print("Field specifications:")
for name, spec in UserProfile._specs.items():
    print(f"  {name}: {spec.annotation}")
    print(f"    nullable={spec.is_nullable}, listable={spec.is_listable}")
    print(f"    description={spec.get('description')}")

Field specifications:
  username: <class 'str'>
    nullable=False, listable=False
    description=User login name
  email: str | None
    nullable=True, listable=False
    description=User email address
  age: int | None
    nullable=True, listable=False
    description=User age in years
  tags: list[str]
    nullable=False, listable=True
    description=User tags


In [37]:
# Generate type annotations for adapters (Pydantic, attrs, etc.)
print("Type annotations for adapter:")
for name, spec in UserProfile._specs.items():
    annotated = spec.annotated()
    print(f"  {name}: {annotated}")

Type annotations for adapter:
  username: typing.Annotated[str, Meta(key='name', value='username'), Meta(key='description', value='User login name'), Meta(key='nullable', value=False)]
  email: typing.Annotated[str | None, Meta(key='name', value='email'), Meta(key='description', value='User email address'), Meta(key='nullable', value=True), Meta(key='validator', value=<function validate_email at 0x10769c4a0>)]
  age: typing.Annotated[int | None, Meta(key='name', value='age'), Meta(key='description', value='User age in years'), Meta(key='nullable', value=True)]
  tags: typing.Annotated[str, Meta(key='name', value='tags'), Meta(key='description', value='User tags'), Meta(key='listable', value=True), Meta(key='default_factory', value=<class 'list'>)]


In [38]:
# Validate using spec validators
def validate_profile(profile: UserProfile) -> UserProfile:
    """Validate profile using Spec validators."""
    errors = []

    # Validate email if present
    if profile.email is not None:
        email_spec = UserProfile._specs["email"]
        validator = email_spec.get("validator")
        if validator is not Undefined:
            try:
                profile.email = validator(profile.email)
            except ValueError as e:
                errors.append(e)

    # Check required fields
    for name, spec in UserProfile._specs.items():
        if not spec.is_nullable:
            value = getattr(profile, name, Undefined)
            if value is Undefined or value is None:
                errors.append(ValueError(f"Required field missing: {name}"))

    if errors:
        raise ExceptionGroup("Profile validation failed", errors)

    return profile


# Valid profile
valid = UserProfile(username="alice", email="alice@example.com", age=30)
validated = validate_profile(valid)
print(f"✓ Valid profile: {validated.username}")

# Invalid email
try:
    invalid = UserProfile(username="bob", email="invalid")
    validate_profile(invalid)
except ExceptionGroup as eg:
    print(f"\n✓ Validation errors: {len(eg.exceptions)}")
    for e in eg.exceptions:
        print(f"  - {e}")

✓ Valid profile: alice

✓ Validation errors: 1
  - Invalid email: invalid


## 11. Advanced Patterns

Common patterns for complex field specifications.

In [39]:
# Pattern 1: Conditional defaults based on other fields
def created_at_factory():
    import datetime as dt

    return dt.datetime.now(dt.UTC)


def updated_at_factory():
    import datetime as dt

    return dt.datetime.now(dt.UTC)


timestamp_specs = {
    "created_at": Spec(
        dt.datetime,
        name="created_at",
        description="Record creation timestamp",
        default_factory=created_at_factory,
    ),
    "updated_at": Spec(
        dt.datetime,
        name="updated_at",
        description="Last update timestamp",
        default_factory=updated_at_factory,
    ),
}

print("Timestamp specs with factories:")
for name, spec in timestamp_specs.items():
    print(f"  {name}: has_factory={spec.has_default_factory}")
    value = spec.create_default_value()
    print(f"    created: {value}")

Timestamp specs with factories:
  created_at: has_factory=True
    created: 2025-11-09 05:55:30.820346+00:00
  updated_at: has_factory=True
    created: 2025-11-09 05:55:30.820361+00:00


In [40]:
# Pattern 2: Range validators
def range_validator(min_val: float, max_val: float):
    """Create range validator closure."""

    def validator(value: float) -> float:
        if not (min_val <= value <= max_val):
            raise ValueError(f"Value {value} out of range [{min_val}, {max_val}]")
        return value

    return validator


temperature_spec = Spec(
    float,
    name="temperature",
    description="LLM temperature parameter",
    default=0.7,
    validator=range_validator(0.0, 2.0),
)

print(f"Temperature spec: {temperature_spec.annotation}")
print(f"Default: {temperature_spec.default}")

# Test range
validator = temperature_spec["validator"]
print(f"\n✓ Valid: {validator(0.5)}")
print(f"✓ Valid: {validator(1.5)}")

try:
    validator(3.0)
except ValueError as e:
    print(f"✓ Invalid: {e}")

Temperature spec: <class 'float'>
Default: 0.7

✓ Valid: 0.5
✓ Valid: 1.5
✓ Invalid: Value 3.0 out of range [0.0, 2.0]


In [41]:
# Pattern 3: Composite validators
def length_validator(min_len: int = 0, max_len: int | None = None):
    """Validate string length."""

    def validator(value: str) -> str:
        if len(value) < min_len:
            raise ValueError(f"Too short (min: {min_len})")
        if max_len is not None and len(value) > max_len:
            raise ValueError(f"Too long (max: {max_len})")
        return value

    return validator


def pattern_validator(pattern: str):
    """Validate against regex pattern."""
    import re

    regex = re.compile(pattern)

    def validator(value: str) -> str:
        if not regex.match(value):
            raise ValueError(f"Does not match pattern: {pattern}")
        return value

    return validator


username_spec = Spec(
    str,
    name="username",
    description="Username (3-20 chars, alphanumeric)",
    validator=[length_validator(min_len=3, max_len=20), pattern_validator(r"^[a-zA-Z0-9_]+$")],
)


# Test composite validation
def validate_all(spec: Spec, value: str) -> str:
    """Run all validators."""
    validators = spec["validator"]
    for validator in validators:
        value = validator(value)
    return value


print(f"✓ Valid: {validate_all(username_spec, 'alice_123')}")

try:
    validate_all(username_spec, "ab")  # Too short
except ValueError as e:
    print(f"✓ Too short: {e}")

try:
    validate_all(username_spec, "alice@domain")  # Invalid chars
except ValueError as e:
    print(f"✓ Invalid chars: {e}")

✓ Valid: alice_123
✓ Too short: Too short (min: 3)
✓ Invalid chars: Does not match pattern: ^[a-zA-Z0-9_]+$


In [42]:
# Pattern 4: Spec registry for reusable field definitions
class SpecRegistry:
    """Registry of common field specifications."""

    # Common specs
    ID = Spec(str, name="id", description="Unique identifier", nullable=False)

    CREATED_AT = Spec(
        dt.datetime,
        name="created_at",
        description="Creation timestamp",
        default_factory=lambda: dt.datetime.now(dt.UTC),
    )

    EMAIL = Spec(
        str, name="email", description="Email address", nullable=True, validator=validate_email
    )

    TAGS = Spec(str, name="tags", description="Tags", listable=True, default_factory=list)


# Use registry specs
print("Registry specs:")
print(f"  ID: {SpecRegistry.ID.annotation}")
print(f"  CREATED_AT: {SpecRegistry.CREATED_AT.annotation}")
print(f"  EMAIL: {SpecRegistry.EMAIL.annotation}")
print(f"  TAGS: {SpecRegistry.TAGS.annotation}")

# Customize registry specs
required_email = SpecRegistry.EMAIL.with_updates(nullable=False)
print(f"\nCustomized: nullable={required_email.is_nullable}")

Registry specs:
  ID: <class 'str'>
  CREATED_AT: <class 'datetime.datetime'>
  EMAIL: str | None
  TAGS: list[str]

Customized: nullable=False


## Summary Checklist

**CommonMeta:**
- ✅ Standard metadata keys (NAME, NULLABLE, LISTABLE, VALIDATOR, DEFAULT, DEFAULT_FACTORY)
- ✅ Metadata validation (no duplicate keys, valid constraints)
- ✅ prepare() method for creating validated metadata tuples

**Spec Construction:**
- ✅ Combines base_type + metadata
- ✅ Multiple construction patterns (kwargs, Meta objects, metadata tuple)
- ✅ Validates base_type is actually a type
- ✅ Supports generic types (list[T], dict[K, V], etc.)
- ✅ Immutable (frozen dataclass)

**Metadata Access:**
- ✅ Dict-like access: `spec['key']` (raises KeyError)
- ✅ Safe access: `spec.get('key', default)` (returns Undefined)
- ✅ Properties: name, is_nullable, is_listable, default
- ✅ Custom metadata keys supported

**Default Values:**
- ✅ Static defaults (default=value)
- ✅ Factory functions (default_factory=callable)
- ✅ Async factory support (with warning)
- ✅ Sync creation: create_default_value()
- ✅ Async creation: acreate_default_value()
- ✅ Cannot have both default and default_factory

**Validators:**
- ✅ Single validator function
- ✅ Multiple validators (list)
- ✅ Validator validation (must be callable)
- ✅ Integration with Operable validation

**Transformations:**
- ✅ with_updates(**kw) - general update
- ✅ as_nullable() - convenience for nullable=True
- ✅ as_listable() - convenience for listable=True
- ✅ with_default(value_or_factory) - add default
- ✅ with_validator(validator) - add validator
- ✅ All transformations return new instances (immutable)
- ✅ Chainable API

**Type Annotations:**
- ✅ annotation property - plain type with nullable/listable modifiers
- ✅ annotated() method - Annotated[type, metadata...]
- ✅ Thread-safe LRU cache for annotated types
- ✅ Nullable uses union syntax (T | None)
- ✅ None/Undefined base_type becomes Any

**Metadata Dict:**
- ✅ metadict() - extract metadata as dict
- ✅ exclude - exclude specific keys
- ✅ exclude_common - exclude CommonMeta keys
- ✅ Combines both exclusion modes

**Integration Patterns:**
- ✅ Operable structured output schemas
- ✅ Adapter type generation (Pydantic, attrs, etc.)
- ✅ Validation framework integration
- ✅ Spec registries for reusable definitions
- ✅ Range validators, composite validators
- ✅ Factory-based conditional defaults

**Next Steps:**
- See `Operable` for structured operation patterns using Spec
- See `Element` for entity base classes
- See framework adapters (Pydantic, attrs) for Spec integration